In [1]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2016, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


In [2]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "pdfs6"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', "2016-6-1", 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification Updates")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 1 of 34
Creating index 6 of 34                    
Creating index 9 of 34                    
Creating index 16 of 34                    
Creating index 22 of 34                    
Creating index 29 of 34                    
Creating index 34 of 34                    
Got update: PGS_IPG_FLAG_WAITING 
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (29/34)
Got update: PGS_IPG_FLAG_WAITING 2
Getting PDF Page Info (34/34)
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 
Got update: PGS_BEGIN_BIND_PDF 
Got update: PGS_DONE 
Done


In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI

In [2]:
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')

In [45]:
issues = api.download_issues(project_id="pbssoftware")

AttributeError: 'int' object has no attribute 'sprint_milestone'

In [48]:
filtered = [i for i in issues.values() if i.sprint_milestone.name == "3.0"]

In [61]:
import re
magic = re.compile(r"[\+\*]+(3.0Web)?(FRS\d+[0-9\.]+)")
magic2 = re.compile(r"(3.0Web)?(FRS\d+[0-9\.]+)")

In [59]:
d=v.description
re.findall("(3.0WebFRS[0-9\.]*)", d)
magic.findall(d)

[]

In [86]:
frs = set()
frs2 = set()

for v in filtered:
    d = v.description or ""
    l = magic.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs.add(result)
    l = magic2.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs2.add(result)
    
sorted(frs)

[(420, 'FRS420.0'),
 (977, 'FRS977.0'),
 (989, 'FRS989.0'),
 (1073, 'FRS1073.01'),
 (1073, 'FRS1073.02'),
 (1073, 'FRS1073.03'),
 (1073, 'FRS1073.11'),
 (1073, 'FRS1073.1304'),
 (1073, 'FRS1073.1305'),
 (1073, 'FRS1073.1306'),
 (1073, 'FRS1073.1307'),
 (1073, 'FRS1073.1308'),
 (1073, 'FRS1073.1309'),
 (1073, 'FRS1073.1310'),
 (1073, 'FRS1073.1311'),
 (1073, 'FRS1073.1312'),
 (1073, 'FRS1073.1313'),
 (1073, 'FRS1073.1314'),
 (1073, 'FRS1073.1315'),
 (1073, 'FRS1073.1316'),
 (1262, 'FRS1262.0'),
 (1263, 'FRS1263.0'),
 (1583, 'FRS1583.0'),
 (1583, 'FRS1583.1'),
 (1583, 'FRS1583.2'),
 (1583, 'FRS1583.3'),
 (1583, 'FRS1583.4'),
 (1754, 'FRS1754.0'),
 (1759, 'FRS1759.0'),
 (1845, 'FRS1845.0'),
 (1902, 'FRS1902.0'),
 (2115, 'FRS2115.0'),
 (2210, 'FRS2210.0'),
 (2303, 'FRS2303.0'),
 (2304, 'FRS2304.1'),
 (2304, 'FRS2304.10'),
 (2304, 'FRS2304.11'),
 (2304, 'FRS2304.12'),
 (2304, 'FRS2304.13'),
 (2304, 'FRS2304.14'),
 (2304, 'FRS2304.15'),
 (2304, 'FRS2304.16'),
 (2304, 'FRS2304.17'),
 (2304, '

In [85]:
frs2-frs

{(1073, 'FRS1073.0'),
 (1073, 'FRS1073.1'),
 (1073, 'FRS1073.13'),
 (3066, '3.0WebFRS001'),
 (3066, '3.0WebFRS002'),
 (3066, '3.0WebFRS119'),
 (3067, '3.0WebFRS003'),
 (3068, '3.0WebFRS028'),
 (3068, '3.0WebFRS036'),
 (3068, '3.0WebFRS037'),
 (3068, '3.0WebFRS067'),
 (3071, '3.0WebFRS004'),
 (3071, '3.0WebFRS005'),
 (3071, '3.0WebFRS006'),
 (3071, '3.0WebFRS120'),
 (3072, '3.0WebFRS007'),
 (3072, '3.0WebFRS008'),
 (3072, '3.0WebFRS009'),
 (3072, '3.0WebFRS010'),
 (3072, '3.0WebFRS011'),
 (3072, '3.0WebFRS012'),
 (3072, '3.0WebFRS013'),
 (3072, '3.0WebFRS120'),
 (3073, '3.0WebFRS015'),
 (3073, '3.0WebFRS016'),
 (3073, '3.0WebFRS017'),
 (3073, '3.0WebFRS018'),
 (3073, '3.0WebFRS019'),
 (3073, '3.0WebFRS020'),
 (3073, '3.0WebFRS021'),
 (3073, '3.0WebFRS022'),
 (3073, '3.0WebFRS023'),
 (3073, '3.0WebFRS024'),
 (3073, '3.0WebFRS025'),
 (3073, '3.0WebFRS026'),
 (3073, '3.0WebFRS027'),
 (3073, '3.0WebFRS028'),
 (3073, '3.0WebFRS029'),
 (3073, '3.0WebFRS030'),
 (3073, '3.0WebFRS031'),
 (3073, 

In [57]:
print(issues[420].description)

*Past Behavior:*
When a table gets corrupted, we get this error when we write to it: LV error code -2147467259 Cannot open database ''. It may not be a database that your application recognizes, or the file may be corrupt. This might be caused by improper shutdown of the database. Running repair on the database with the built in windows tools solves this issue

*Expected Behavior:*
*FRS420.0*
Before writing data to the database, check to make sure there are no NaN values of invalid characters. If there are NaN, replace with -9999999 and replace Inf with 9999999.






In [87]:
def extract_frs_items():
    magic = re.compile(r"[\+\*]+(3.0Web)?(FRS\d+[0-9\.]+)")
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    iss = api.download_issues(project_id="pbssoftware")
    filtered = [i for i in iss.values() if i.sprint_milestone.name == "3.0"]
    
    reqs = []
    for v in filtered:
        d = v.description or ""
        matches = magic.findall(d)
        for web, fr in matches:
            web = web or ""
            result = v.id, web + fr
            reqs.append(result)
    return sorted(reqs)

In [88]:
extract_frs_items()

[(420, 'FRS420.0'),
 (977, 'FRS977.0'),
 (989, 'FRS989.0'),
 (1073, 'FRS1073.01'),
 (1073, 'FRS1073.02'),
 (1073, 'FRS1073.03'),
 (1073, 'FRS1073.11'),
 (1073, 'FRS1073.1304'),
 (1073, 'FRS1073.1305'),
 (1073, 'FRS1073.1306'),
 (1073, 'FRS1073.1307'),
 (1073, 'FRS1073.1308'),
 (1073, 'FRS1073.1309'),
 (1073, 'FRS1073.1310'),
 (1073, 'FRS1073.1311'),
 (1073, 'FRS1073.1312'),
 (1073, 'FRS1073.1313'),
 (1073, 'FRS1073.1314'),
 (1073, 'FRS1073.1315'),
 (1073, 'FRS1073.1316'),
 (1262, 'FRS1262.0'),
 (1263, 'FRS1263.0'),
 (1583, 'FRS1583.0'),
 (1583, 'FRS1583.1'),
 (1583, 'FRS1583.2'),
 (1583, 'FRS1583.3'),
 (1583, 'FRS1583.4'),
 (1754, 'FRS1754.0'),
 (1759, 'FRS1759.0'),
 (1845, 'FRS1845.0'),
 (1902, 'FRS1902.0'),
 (2115, 'FRS2115.0'),
 (2210, 'FRS2210.0'),
 (2303, 'FRS2303.0'),
 (2304, 'FRS2304.1'),
 (2304, 'FRS2304.10'),
 (2304, 'FRS2304.11'),
 (2304, 'FRS2304.12'),
 (2304, 'FRS2304.13'),
 (2304, 'FRS2304.14'),
 (2304, 'FRS2304.15'),
 (2304, 'FRS2304.16'),
 (2304, 'FRS2304.17'),
 (2304, '